In [42]:
from sqlalchemy import create_engine  # to interact with postgresql
import pyodbc  # to connect to sqlserver
import pandas as pd  # data extract and load
import os  # for the stored username and password in the system environment variables


In [43]:
# get the password from environment variables

pwd = os.environ['PGPASS']
uid = os.environ['PGUID']


In [48]:
# sqlserver db details
driver = "{SQL Server}"
server = "DESKTOP-HD9D72O"
database = "AdventureWorksDW2019"


In [49]:
# extract data from sql server
try:
    cnxn = pyodbc.connect(
        'DRIVER=' + driver + ';SERVER =' + server + '\SQLEXPRESS'+ ';DATABASE =' + database + ';UID=' + uid + ';PWD==' + pwd)
    cursor = cnxn.cursor()
except Exception as e:
    print("Connection error" + str(e))

def extract():
    try:
        # execute query
        cnxn.execute("""select t.name as table_name\
            from sys.tables to where t.name in ('DimProduct', 'DimProductCategory', 'DimProductSubCategory', 'DimSalesTerritory')""")
        scr_tables = cnxn.cursor.fetchall()
        for tbl in scr_tables:
            # query and load save data to dataframe
            df = pd.read_sql_query(f'select * from {tbl[0]}', src_conn)
            load(df, tbl[0])
    except Exception as e:
        print("Data extract error: " + str(e))
    finally:
        cnxn.close()


Connection error('08001', '[08001] [Microsoft][ODBC SQL Server Driver]Neither DSN nor SERVER keyword supplied (0) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver]Invalid connection string attribute (0)')


In [33]:
# load data to postgres


def load(df, tbl):
    try:
        rowes_imported = 0
        engine = create_engine(
            f'postgresql://{uid}:{pwd}@{server}:5432/AdventureWorks')
        print(
            f'importing rows {rows_imported} to {rows_imported + len(df) } ... for table {tbl}')
        # save df to postres
        df.to_sql(f'stg_{tbl}', engine, if_exists='replace', index=False)
        rows_imported += len(df)
        # print success message
        print("Data import was successful")
    except Exception as e:
        print("Data load error: " + str(e))


In [34]:
 try:
    # call extract function
    extract()
except Exception as e:
    print("Error while extracting data: " + str(e))

# try:
#     # call load function
#     load(df, tbl)
# except Exception as e:
#     print("Error while loading data: " + str(e))

Data extract error: ('08001', '[08001] [Microsoft][SQL Server Native Client 11.0]Neither DSN nor SERVER keyword supplied (0) (SQLDriverConnect); [08001] [Microsoft][SQL Server Native Client 11.0]Invalid connection string attribute (0)')
Error while extracting data: local variable 'cnxn' referenced before assignment
